In [ ]:
import os
import numpy as np
import pandas as pd
import eureka.lib.plots
import eureka.S3_data_reduction.s3_reduce as s3
import eureka.S4_generate_lightcurves.s4_genLC as s4
# import eureka.S5_lightcurve_fitting.s5_fit as s5
import eureka.S5_lightcurve_fitting.s5_fit_GA as s5
import eureka.S6_planet_spectra.s6_spectra as s6
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import astropy.io.fits as pf
# import scipy.interpolate as spi
# import scipy.ndimage.interpolation as spni
# import multiprocessing as mp
import eureka.lib.sort_nicely as sn 
import eureka.lib.smoothing as sm
import eureka.S3_data_reduction.hst_scan as hst
import eureka.S3_data_reduction.QL_params as evp
import eureka.S3_data_reduction.sigrej
import eureka.S3_data_reduction.optspex
import importlib, shutil, re
import pickle
# import json
from datetime import datetime

importlib.reload(hst)
importlib.reload(evp)

from eureka.lib import readECF
from eureka.lib import readEPF
from eureka.lib import exomast
from eureka.lib import generate_epf
from eureka.lib import optimizers
from eureka.lib import objective_funcs

In [ ]:
"""
Eureka! Optimization for HST WFC-3 G141 Data
---------------------------------------------

Description:
This script is designed to optimize the parameters used in HST WFC-3 G141 data analysis. 
The optimization utilizes both parametric sweeps and a genetic algorithm (GA) in tandem with MCMC optimization techniques. 

Sections:
1. IMPORT DEPENDENCIES

2. AUTOMATION CONFIGURATION (STEP 0):
    - Configuration of the automation involves steps like window size retrieval, loading ECFs, generating new EPFs from MAST data, and cropping the xwindow.

3. PARAMETRIC OPTIMIZATION (STEP 1.0):
    - If selected by the user, the script will conduct a parametric sweep for optimizing S3-S4 inputs.

4. GENETIC ALGORITHM (STEP 1.1):
    - If selected by the user, the script will employ a GA to optimize the parameters. The GA will run over multiple "Steps" or phases, each having its population size and generation count.

5. SAVE OPTIMIZED VALUES (STEP 1.2):
    - The script saves the optimal ECF values obtained from the previous optimizations.

6. MCMC OPTIMIZATION (STEP 1.3):
    - The script performs an MCMC optimization for EPF values.

7. PROCESS WHITE LIGHT DATA (STEP 1.3):
    - The script processes white light data by running stages 3-5.

8. PROCESS SPECTROSCOPIC DATA (STEP 2):
    - The script processes spectroscopic data for HST, specifically for 27 channels ranging from 1.12-1.66 um.

Inputs:
- General Parameters like the name of the exoplanet and event label.
- Parametric Optimization Flag and Genetic Algorithm settings, including target fitness and generation settings for different phases.
- File paths for observation data, MAST data, exotic-ld ancillary files, throughput files, and other related files.

Outputs:
- Optimized parameter values saved in the "best" dictionary.

Usage:
Begin by configuring the 'optimizer_inputs.txt' with the relevant parameters and file paths. After configuring, run the script, and the selected optimization method(s) will be executed. Ensure you have all required data and supporting files in the specified paths before running.

Note:
Ensure that the genetic algorithm's population sizes are multiples of 2, as required by the algorithm's design. Additionally, always ensure that the specified file paths are accurate and point to the correct files.
!!! HST ONLY --> Make sure the horizons file selected covers the dates of the observation. !!!

Author: Reza Ashtari
Date: 08/22/2023

"""

In [ ]:
"""
Notes for Improvement:
-----------------------

Code Duplication: Remove redundant / repeated code, especially in the parametric optimization functions. This can be reduced for cleaner code and easier maintenance.

Mutation Strategy: The mutation function currently introduces a random float mutation to an integer gene, which is then rounded back to an integer. Depending on the problem, there may be more effective ways to introduce mutation.

Dynamic Population: Potentially shrink population sizes if solution is converging.

"If main" block: Current usage of "if __name__ == '__main__':" requires revision 

Plotting Strategy: For long-running GAs, constantly updating a live plot can be computationally expensive. Depending on the use case, it might be better to update the plot less frequently.

"""

In [ ]:
## TABLE OF CONTENTS ##

# Eureka! Optimization for HST WFC-3 G141 Data
# USER - Sections of run script where user input is required - Input parameters, Filepaths, Bounds for parameters to be optimized
# STEP 0 - Configure Automation - Retrieve window sizes (HST Only), Load ECF's, Generate new EPF's using MAST data, Crop xwindow
# STEP 1.0 - Parametric Sweeps
# STEP 1.1 - Genetic Algorithms
# STEP 1.2 - Save Optimized ECF Values
# STEP 1.3 - Run Stages 3-5 for White Light Data
# STEP 2 - Run Stages 3-6 for Spectroscopic Data (HST = 27 channels, 1.12-1.66 um)

In [ ]:
## STEP 0 - Load optimizer parameters from input .txt file
optimizer_params = optimizers.read_inputs('optimizer_inputs.txt')

# # Set each parameter as a local variable
# for k, v in optimizer_params.items():
#     locals()[k] = v

# Set each parameter as a local variable (Explicit)
planet_name = optimizer_params['planet_name']
eventlabel = optimizer_params['eventlabel']
bright_star = optimizer_params['bright_star']
xwindow_selection = optimizer_params['xwindow_selection']
ywindow_selection = optimizer_params['ywindow_selection']
spec_hw_selection = optimizer_params['spec_hw_selection']
bg_hw_selection = optimizer_params['bg_hw_selection']
optimizer = optimizer_params['optimizer']
target_fitness = optimizer_params['target_fitness']
population_size_Step_1 = optimizer_params['population_size_Step_1']
generations_Step_1 = optimizer_params['generations_Step_1']
population_size_Step_2 = optimizer_params['population_size_Step_2']
generations_Step_2 = optimizer_params['generations_Step_2']
scaling_chi2red = optimizer_params['scaling_chi2red']
scaling_MAD_spec = optimizer_params['scaling_MAD_spec']
scaling_MAD_white = optimizer_params['scaling_MAD_white']
skip_xwindow_crop = optimizer_params['skip_xwindow_crop']
skip_ywindow_crop = optimizer_params['skip_ywindow_crop']
enable_exotic_ld = optimizer_params['enable_exotic_ld']
clip_first_orbit = optimizer_params['clip_first_orbit']
# clip_orbits = optimizer_params['clip_orbits']
manual_clip_npoints = optimizer_params['manual_clip_npoints']

bounds_xwindow_crop = optimizer_params['bounds_xwindow_crop']
if bounds_xwindow_crop == 'auto':
    bounds_xwindow_crop = [0, 25]

bounds_ywindow_crop = optimizer_params['bounds_ywindow_crop']
if bounds_ywindow_crop == 'auto':
    bounds_ywindow_crop = [0, 40]

bounds_diffthresh = optimizer_params['bounds_diffthresh']
if bounds_diffthresh == 'auto':
    bounds_diffthresh = [3, 10]

bounds_bg_hw = optimizer_params['bounds_bg_hw'] 
bounds_spec_hw = optimizer_params['bounds_spec_hw']

bounds_p3thresh = optimizer_params['bounds_p3thresh']
if bounds_p3thresh == 'auto':
    bounds_p3thresh = [3, 7]

bounds_median_thresh = optimizer_params['bounds_median_thresh']
if bounds_median_thresh == 'auto':
    bounds_median_thresh = [3, 7]

bounds_window_len = optimizer_params['bounds_window_len']
if bounds_window_len == 'auto':
    bounds_window_len = [1, 21]

bounds_p5thresh = optimizer_params['bounds_p5thresh']
if bounds_p5thresh == 'auto':
    bounds_p5thresh = [5, 10]

bounds_p7thresh = optimizer_params['bounds_p7thresh']
if bounds_p7thresh == 'auto':
    bounds_p7thresh = [5, 10]

bounds_drift_range = optimizer_params['bounds_drift_range']
if bounds_drift_range == 'auto':
    bounds_drift_range = [8, 12]

bounds_highpassWidth = optimizer_params['bounds_highpassWidth']
if bounds_highpassWidth == 'auto':
    bounds_highpassWidth = [8, 12]

bounds_sigma = optimizer_params['bounds_sigma']
if bounds_sigma == 'auto':
    bounds_sigma = [8, 12]

bounds_box_width = optimizer_params['bounds_box_width']
if bounds_box_width == 'auto':
    bounds_box_width = [8, 12]

outputdir_optimization = optimizer_params['outputdir_optimization']
loc_sci = optimizer_params['loc_sci']
exomast_file = optimizer_params['exomast_file']
exotic_ld_direc = optimizer_params['exotic_ld_direc']
exotic_ld_file = optimizer_params['exotic_ld_file']
ld_file = optimizer_params['ld_file']
ld_file_white = optimizer_params['ld_file_white']
use_generate_ld = optimizer_params['use_generate_ld']

# # Set conditional inputs
# if xwindow_selection == 'manual':
#     skip_xwindow_crop = True
# if ywindow_selection == 'manual':
#     skip_ywindow_crop = True

In [ ]:
## STEP 0 - Create "best" Dictionary to save optimized values ##
best = {}

# Create dictionaries to keep track of optimization metrics
history_MAD_white = {}
history_MAD_spec = {}
history_MAD_chi2red = {}
history_fitness_score = {}

In [ ]:
## STEP 0 - HST ONLY - Retrieve xwindow & ywindow ##

# Initialize event object
ev  = evp.event_init()

# Object
ev.obj_list = []
ev.img_list = []
# Retrieve all files from science directory
for fname in os.listdir(loc_sci):
    if fname.endswith("ima.fits"):
        filedir     = loc_sci +'/'+ fname
        header      = pf.getheader(filedir)
        if header['OBSTYPE'] == 'SPECTROSCOPIC':
            ev.obj_list.append(filedir)
        elif header['OBSTYPE'] == 'IMAGING':
            ev.img_list.append(filedir)
ev.obj_list = sn.sort_nicely(ev.obj_list) 
ev.img_list = sn.sort_nicely(ev.img_list) 
ev.n_files  = len(ev.obj_list)
ev.n_img    = len(ev.img_list)

# Determine image size, filter/grism, scan height
hdulist         = pf.open(ev.obj_list[0])
nx              = hdulist['SCI',1].header['NAXIS1']
ny              = hdulist['SCI',1].header['NAXIS2']
ev.grism        = hdulist[0].header['FILTER']
ev.detector     = hdulist[0].header['DETECTOR']
ev.flatoffset   = [-1*hdulist['SCI',1].header['LTV2'], -1*hdulist['SCI',1].header['LTV1']]
# ev.n_reads      = hdulist['SCI',1].header['SAMPNUM']
ev.eventdir     = hdulist[0].header['ROOTNAME'][:6]
# scanheight      = hdulist[0].header['SCAN_LEN']/0.121   #Pixels
# ev.spec_width   = np.round(scanheight/2./ev.n_reads+6).astype(int) # Commented Out - RA2023
# ev.fitbghw      = np.round(scanheight/2./ev.n_reads+6).astype(int) # Commented Out - RA2023

# Updates from 11-30-2023
ev.scanrate     = hdulist[0].header['SCAN_RAT']
ev.n_reads      = hdulist[0].header['NSAMP'] - 1
ev.scanheight   = ev.scanrate/0.121*hdulist[0].header['EXPTIME']
ev.spec_width   = np.round(ev.scanheight/2./(ev.n_reads-1)+6).astype(int)
ev.fitbghw      = ev.spec_width

# Determine extraction box location 
data            = hdulist['SCI',1].data 
smdata          = sm.smoothing(data, [5,5])
ydiff           = np.diff(np.sum(smdata,axis=1))[20:-20]  
xdiff           = np.diff(np.sum(smdata,axis=0))[20:-20]   

if bright_star is False:
    ev.ywindow         = [np.argmax(ydiff), np.argmin(ydiff)+40]
if bright_star is True:
    # Modified ev.window for bright targets
    ywindow_min     = max(1, np.argmax(ydiff) - 40)  # Make sure smallest value is not less than the 1st pixel
    ev.ywindow      = [ywindow_min, np.argmin(ydiff) + 85]

if ev.grism == 'G141':
    ev.xwindow         = [np.argmax(xdiff), np.argmin(xdiff)+40]  # Added to ev object returned as output - RA2023
    # ev.xwindow         = [np.argmax(xdiff)-5, np.argmin(xdiff)+45]  # Added to ev object returned as output - RA2023
else:
    # G102 grism doesn't have a sharp cutoff on the blue edge
    ev.xwindow         = [np.argmin(xdiff)-145, np.argmin(xdiff)+40]  # Added to ev object returned as output - RA2023
hdulist.close()

print(f"ev.xwindow = {ev.xwindow}")
print(f"ev.ywindow = {ev.ywindow}")

best['xwindow_LB'] = ev.xwindow[0]
best['xwindow_UB'] = ev.xwindow[1]
best['ywindow_LB'] = ev.ywindow[0]
best['ywindow_UB'] = ev.ywindow[1]

In [ ]:
## STEP 0 - Define Bounds for Parametric and Genetic Optimization ##

bounds_xwindow_LB = [ev.xwindow[0] + bounds_xwindow_crop[0], ev.xwindow[0] + bounds_xwindow_crop[1]]  
bounds_xwindow_UB = [ev.xwindow[1] - bounds_xwindow_crop[1], ev.xwindow[1] - bounds_xwindow_crop[0]]  

bounds_ywindow_LB = [ev.ywindow[0] + bounds_ywindow_crop[0], ev.ywindow[0] + bounds_ywindow_crop[1]]  
bounds_ywindow_UB = [ev.ywindow[1] - bounds_ywindow_crop[1], ev.ywindow[1] - bounds_ywindow_crop[0]]  


# Automate bg_hw & spec_hw bounds estimates 

# if bounds_bg_hw == 'auto' or bounds_spec_hw == 'auto':
#         bounds_bg_hw_LB = max(ev.spec_width - 5, 0)
#         bounds_bg_hw_UB = ev.spec_width + 5
#         bounds_bg_hw = [bounds_bg_hw_LB, bounds_bg_hw_UB]
#         bounds_spec_hw = bounds_bg_hw

# if bright_star is True:
#     if bounds_bg_hw == 'auto' or bounds_spec_hw == 'auto':
#             bounds_bg_hw_LB = max(ev.spec_width - 10, 0)
#             bounds_bg_hw_UB = ev.spec_width + 10
#             bounds_bg_hw = [bounds_bg_hw_LB, bounds_bg_hw_UB]
#             bounds_spec_hw = bounds_bg_hw

if bounds_bg_hw == 'auto' or bounds_spec_hw == 'auto':
        bounds_bg_hw_LB = max(ev.spec_width - 10, 1)
        bounds_bg_hw_UB = ev.spec_width + 10
        bounds_bg_hw = [bounds_bg_hw_LB, bounds_bg_hw_UB]
        bounds_spec_hw = bounds_bg_hw


# Define bounds for inputs used in Genetic Algorithm
min_bounds = np.array([
                        bounds_diffthresh[0],
                        bounds_bg_hw[0], 
                        bounds_spec_hw[0], 
                        bounds_p3thresh[0], 
                        bounds_median_thresh[0],
                        bounds_window_len[0], 
                        bounds_p5thresh[0], 
                        bounds_p7thresh[0],
                        bounds_drift_range[0], 
                        bounds_highpassWidth[0], 
                        bounds_sigma[0], 
                        bounds_box_width[0]])

max_bounds = np.array([
                        bounds_diffthresh[1],
                        bounds_bg_hw[1], 
                        bounds_spec_hw[1], 
                        bounds_p3thresh[1], 
                        bounds_median_thresh[1], 
                        bounds_window_len[1], 
                        bounds_p5thresh[1], 
                        bounds_p7thresh[1],
                        bounds_drift_range[1], 
                        bounds_highpassWidth[1], 
                        bounds_sigma[1], 
                        bounds_box_width[1]])


# # Define bounds for inputs used in Step 1 of GA
# min_bounds_Step_1 = np.array([bounds_bg_hw[0], 
#                         bounds_spec_hw[0]])

# max_bounds_Step_1 = np.array([bounds_bg_hw[1], 
#                         bounds_spec_hw[1]])


# Define bounds for inputs used in Step 1 of GA
min_bounds_Step_1 = np.array([bounds_p3thresh[0], 
                        bounds_median_thresh[0],
                        bounds_window_len[0], 
                        bounds_p5thresh[0], 
                        bounds_p7thresh[0]])

max_bounds_Step_1 = np.array([bounds_p3thresh[1], 
                        bounds_median_thresh[1], 
                        bounds_window_len[1], 
                        bounds_p5thresh[1], 
                        bounds_p7thresh[1]])


# Define bounds for inputs used in Step 2 of GA
min_bounds_Step_2 = np.array([bounds_drift_range[0], 
                        bounds_highpassWidth[0], 
                        bounds_sigma[0], 
                        bounds_box_width[0]])

max_bounds_Step_2 = np.array([bounds_drift_range[1], 
                        bounds_highpassWidth[1], 
                        bounds_sigma[1], 
                        bounds_box_width[1]])

In [ ]:
## STEP 0 - Load ECF Files ## 

eureka.lib.plots.set_rc(style='eureka', usetex=False, filetype='.png')

ecf_path = '.'+os.sep

# Load Eureka! control file and store values in Event object
s3_ecffile = 'S3_' + eventlabel + '.ecf'
s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)

s4_ecffile = 'S4_' + eventlabel + '.ecf'
s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)

s5_ecffile = 'S5_' + eventlabel + '.ecf'
s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

s6_ecffile = 'S6_' + eventlabel + '.ecf'
s6_meta_GA = readECF.MetaClass(ecf_path, s6_ecffile)


# # Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
# s3_meta_GA.xwindow = ev.xwindow
# s3_meta_GA.ywindow = ev.ywindow


# Set xwindow and ywindow values for auto vs. manual box extraction
if xwindow_selection == 'auto':
    s3_meta_GA.xwindow = ev.xwindow
    best['xwindow_LB'] = ev.xwindow[0]
    best['xwindow_UB'] = ev.xwindow[1]

if xwindow_selection == 'manual':
    ev.xwindow = s3_meta_GA.xwindow
    best['xwindow_LB'] = ev.xwindow[0]
    best['xwindow_UB'] = ev.xwindow[1]

if ywindow_selection == 'auto':
    s3_meta_GA.ywindow = ev.ywindow
    best['ywindow_LB'] = ev.ywindow[0]
    best['ywindow_UB'] = ev.ywindow[1]

if ywindow_selection == 'manual':
    ev.ywindow = s3_meta_GA.ywindow
    best['ywindow_LB'] = ev.ywindow[0]
    best['ywindow_UB'] = ev.ywindow[1]
    
# Set spec_hw & bg_hw values for auto vs. manual selection methods
if spec_hw_selection == 'auto':
    s3_meta_GA.spec_hw = ev.spec_width
if bg_hw_selection == 'auto':
    s3_meta_GA.bg_hw = ev.spec_width

In [ ]:
## STEP 0 - Load ExoMAST Data ##

# Load the data from the pickle file
with open(exomast_file, 'rb') as f:
    all_planet_data = pickle.load(f)

# Query the locally saved catalog for an exoplanet's data
if planet_name in all_planet_data:
    planet_data = all_planet_data[planet_name]
    print(planet_data)
else:
    print(f"No offline data found for {planet_name}. Retrieving planetary data from exoMAST (ONLINE)")
    planet_name = eventlabel
    planet_data = exomast.get_target_data(planet_name)[0]
# else:
#     print(f"No data found for {planet_name}")

In [ ]:
## STEP 0 - Generate EPF from ExoMAST Data ##

# # Generate EPF
directory = s4_meta_GA.topdir
if not os.path.exists(directory):
    os.makedirs(directory)

s5_epffile = 's5_fit_par_' + eventlabel + '.epf'
    
# Call the Generate EPF function
generate_epf.start(planet_data, file_path=os.path.join(directory, s5_epffile))

# Load params object          
params = readEPF.Parameters(s5_meta_GA.folder, s5_meta_GA.fit_par)

In [ ]:
## STEP 0 - Initial Run ##

print("Initial Run.")

s3_spec, s3_meta = s3.reduce(eventlabel, input_meta=s3_meta_GA)
s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, input_meta=s4_meta_GA, s3_meta=s3_meta)

# Calculate which data points to be clipped (set in optimizer_inputs.txt)
manual_clip_lists = []

if clip_first_orbit:
    zero_indices = np.where(s4_meta.orbitnum == 0)
    maxIndex_firstOrbit = max(zero_indices[0])
    manual_clip_lists.append([0, maxIndex_firstOrbit + 1])

# if clip_orbits is not None:
#     for orbit_num in clip_orbits:
#         orbit_indices = np.where(s4_meta.orbitnum == orbit_num)
#         if orbit_indices[0].size > 0:
#             max_index = max(orbit_indices[0])
#             manual_clip_lists.append([min(orbit_indices[0]), max_index + 1])

firstFrames = np.where(s4_meta.framenum == 0)
firsts = list(firstFrames[0][1:])

subtract_value = 0  # Clip first data point in each frame
# subtract_value = 3  # Clip first 4 data points of each frame

npoints = manual_clip_npoints-1  # Clip first n data points of each frame

for item in firsts:
    subtract_value += 0
    item -= subtract_value
    # manual_clip_lists.append([item, item+1])  # Clip first data point in each frame
    manual_clip_lists.append([item, item+npoints])  # Clip first n data points of each frame

print(manual_clip_lists)

s5_meta_GA.inputdir = s4_meta.outputdir
s5_meta_GA.fit_method = 'lsq'

params = readEPF.Parameters(s5_meta_GA.folder, s5_meta_GA.fit_par)

s5_meta_GA.manual_clip = manual_clip_lists
s5_meta = s5.fitlc(eventlabel, params, input_meta=s5_meta_GA, s4_meta=s4_meta)

# Save Metrics
history_MAD_white['Initial_Run'] = (sum(s4_meta.mad_s4_binned) / len(s4_meta.mad_s4_binned))
history_MAD_spec['Initial_Run'] = s4_meta.mad_s4
history_MAD_chi2red['Initial_Run'] = s5_meta.chi2red

best_fitness_value = (
    scaling_chi2red * s5_meta.chi2red +
    scaling_MAD_spec * s4_meta.mad_s4 +
    scaling_MAD_white *
    (sum(s4_meta.mad_s4_binned) / len(s4_meta.mad_s4_binned))
)

history_fitness_score['Initial_Run'] = best_fitness_value

In [ ]:
## STEP 0 - Parametric Sweep of "ywindow_crop" ##

if skip_ywindow_crop is False:

    print("Performing ywindow cropping.")

    ## Setup Meta ##
    s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
    s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)

    # Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
    s3_meta_GA.xwindow = ev.xwindow
    s3_meta_GA.ywindow = ev.ywindow

    # Set spec_hw & bg_hw values for auto vs. manual selection methods
    if spec_hw_selection == 'auto':
        s3_meta_GA.spec_hw = ev.spec_width
    if bg_hw_selection == 'auto':
        s3_meta_GA.bg_hw = ev.spec_width

    best_param, best_fitness_value = optimizers.parametric_sweep_ywindow_crop(objective_funcs.ywindow_crop, bounds_ywindow_crop, eventlabel, ev, s3_meta_GA, s4_meta_GA)

    best['ywindow_LB'] = ev.ywindow[0] + best_param[0]
    best['ywindow_UB'] = ev.ywindow[1] - best_param[0]

    print("Best parameter: ", best_param[0])
    print("Best fitness: ", best_fitness_value)

    ev.ywindow = [best['ywindow_LB'], best['ywindow_UB']]
    
    s3_meta_GA.ywindow = ev.ywindow

    print(s3_meta_GA.ywindow)

    history_fitness_score['ywindow_crop'] = best_fitness_value

In [ ]:
## STEP 0 - Skip ywindow cropping ##

if skip_ywindow_crop is True:
    
    best['ywindow_LB'] = ev.ywindow[0] 
    best['ywindow_UB'] = ev.ywindow[1] 
    
    s3_meta_GA.ywindow = ev.ywindow
    
    print(f"s3_meta_GA.ywindow = {s3_meta_GA.ywindow}")

In [ ]:
## STEP 0 - Crop xwindow - Determine pixels for min & max GRISM wavelengths ##

if skip_xwindow_crop is False:

    # Run Stage 3 & Stage 4 Once
    if __name__ == '__main__':
        s3_spec, s3_meta = s3.reduce(eventlabel, input_meta=s3_meta_GA)
        s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, input_meta=s4_meta_GA, s3_meta=s3_meta)

    # Find pixels matching wave_min & wave_max
    def find_closest_index(array, value):
        return np.abs(array - value).argmin()

    index_wave_min = find_closest_index(s4_spec.wave_1d, s4_meta_GA.wave_min)
    index_wave_max = find_closest_index(s4_spec.wave_1d, s4_meta_GA.wave_max)

    pixel_wave_min = int(s4_spec.x[index_wave_min].values)
    pixel_wave_max = int(s4_spec.x[index_wave_max].values)

    print("wave_min pixel: ", pixel_wave_min)
    print("wave_max pixel: ", pixel_wave_max)

    print(ev.xwindow)

    s3_meta_GA.xwindow = ev.xwindow
    s3_meta_GA.ywindow = ev.ywindow

In [ ]:
## STEP 0 - Parametric Sweep of "xwindow_crop" ##

if skip_xwindow_crop is False:

    print("Performing xwindow cropping.")

    ## Setup Meta ##
    s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
    s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)

    # Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
    s3_meta_GA.xwindow = ev.xwindow
    s3_meta_GA.ywindow = ev.ywindow

    # Set spec_hw & bg_hw values for auto vs. manual selection methods
    if spec_hw_selection == 'auto':
        s3_meta_GA.spec_hw = ev.spec_width
    if bg_hw_selection == 'auto':
        s3_meta_GA.bg_hw = ev.spec_width

    best_param, best_fitness_value = optimizers.parametric_sweep_xwindow_crop(objective_funcs.xwindow_crop, bounds_xwindow_crop, eventlabel, ev, pixel_wave_min, pixel_wave_max, s3_meta_GA, s4_meta_GA)

    best['xwindow_LB'] = ev.xwindow[0] + best_param[0]
    best['xwindow_UB'] = ev.xwindow[1] - best_param[0]

    print("Best parameter: ", best_param[0])
    print("Best fitness: ", best_fitness_value)

    ev.xwindow = [best['xwindow_LB'], best['xwindow_UB']]
    
    s3_meta_GA.xwindow = ev.xwindow

    print(s3_meta_GA.xwindow)

    history_fitness_score['xwindow_crop'] = best_fitness_value

In [ ]:
## STEP 0 - Skip xwindow cropping ##

if skip_xwindow_crop is True:
    
    best['xwindow_LB'] = ev.xwindow[0] 
    best['xwindow_UB'] = ev.xwindow[1] 
    
    s3_meta_GA.xwindow = ev.xwindow
    
    print(f"s3_meta_GA.xwindow = {s3_meta_GA.xwindow}")

In [ ]:
# ## STEP 0 - Manual Clipping - Prerequisite S3 & S4 Runs (Revise once Eureka! bug is updated) ##

# print("Determining manual clipping lists.")

# if __name__ == '__main__':
#     s3_spec, s3_meta = s3.reduce(eventlabel, input_meta=s3_meta_GA)
#     s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, input_meta=s4_meta_GA, s3_meta=s3_meta)

In [ ]:
# ## STEP 0 - Manual Clipping (Revise once Eureka! bug is updated) ##

# manual_clip_lists = []

# if clip_first_orbit:
#     zero_indices = np.where(s4_meta.orbitnum == 0)
#     maxIndex_firstOrbit = max(zero_indices[0])
#     manual_clip_lists.append([0, maxIndex_firstOrbit + 1])

# # if clip_orbits is not None:
# #     for orbit_num in clip_orbits:
# #         orbit_indices = np.where(s4_meta.orbitnum == orbit_num)
# #         if orbit_indices[0].size > 0:
# #             max_index = max(orbit_indices[0])
# #             manual_clip_lists.append([min(orbit_indices[0]), max_index + 1])

# firstFrames = np.where(s4_meta.framenum == 0)
# firsts = list(firstFrames[0][1:])

# subtract_value = 0 # Clip first data point in each frame
# # subtract_value = 3  # Clip first 4 data points of each frame

# npoints = manual_clip_npoints-1 # Clip first n data points of each frame

# for item in firsts:
#     subtract_value += 0
#     item -= subtract_value
#     # manual_clip_lists.append([item, item+1]) # Clip first data point in each frame
#     manual_clip_lists.append([item, item+npoints])  # Clip first n data points of each frame

# print(manual_clip_lists)

# s5_meta_GA.inputdir = s4_meta.outputdir
# s5_meta_GA.fit_method = 'lsq'

# params = readEPF.Parameters(s5_meta_GA.folder, s5_meta_GA.fit_par)

# s5_meta_GA.manual_clip = manual_clip_lists
# s5_meta = s5.fitlc(eventlabel, params, input_meta=s5_meta_GA, s4_meta=s4_meta)

In [ ]:
## STEP 1.0 - Parametric Sweep of "diffthresh" ##

## Setup Meta ##
s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
s3_meta_GA.xwindow = ev.xwindow
s3_meta_GA.ywindow = ev.ywindow

# Set spec_hw & bg_hw values for auto vs. manual selection methods
if spec_hw_selection == 'auto':
    s3_meta_GA.spec_hw = ev.spec_width
if bg_hw_selection == 'auto':
    s3_meta_GA.bg_hw = ev.spec_width

# Set Up Manual Clipping
s5_meta_GA.manual_clip = manual_clip_lists

# Set Fit Method
s5_meta_GA.fit_method = 'lsq'  


# Perform parametric sweep
best_param, best_fitness_value = optimizers.parametric_sweep_S3(objective_funcs.diffthresh, bounds_diffthresh, eventlabel, s3_meta_GA, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

best['diffthresh'] = best_param[0]

print("Best parameters: ", best_param[0])
print("Best fitness: ", best_fitness_value)

history_fitness_score['diffthresh'] = best_fitness_value

In [ ]:
## STEP 1.0 - Parametric Sweep of "bg_hw" & "spec_hw" ##

if optimizer == "parametric" or optimizer == "genetic":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
	s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
	s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

	# Set spec_hw & bg_hw values for auto vs. manual selection methods
	if spec_hw_selection == 'auto':
		s3_meta_GA.spec_hw = ev.spec_width
	if bg_hw_selection == 'auto':
		s3_meta_GA.bg_hw = ev.spec_width

    # Set Up Manual Clipping
	s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
	s5_meta_GA.fit_method = 'lsq'  


    # Setup Meta / Define Initial Population
    # Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
    # Stage 4
	s4_meta_GA.diffthresh = best['diffthresh']
    # Stage 5
	s5_meta_GA.diffthresh = best['diffthresh']


	# Perform parametric sweep
	best_params, best_fitness_value = optimizers.parametric_sweep_double(objective_funcs.bg_hw_spec_hw, bounds_bg_hw, bounds_spec_hw, eventlabel, s3_meta_GA, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

	# Save Results in "best" Dictionary 
	best['bg_hw'] = best_params[0]
	best['spec_hw'] = best_params[1]

	# Print Results of Parametric Sweep
	print("Best parameters: ", best_params)
	print("Best fitness: ", best_fitness_value)

	history_fitness_score['spec_hw_bg_hw'] = best_fitness_value

In [ ]:
## STEP 1.0 - Parametric Sweep of "p3thresh" ##

if optimizer == "parametric":

    ## Setup Meta ##
    s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
    s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
    s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

    # Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
    s3_meta_GA.xwindow = ev.xwindow
    s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
    s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
    s5_meta_GA.fit_method = 'lsq'  


    # Setup Meta / Define Initial Population
    # Stage 3
    s3_meta_GA.diffthresh = best['diffthresh']
    s3_meta_GA.bg_hw = best['bg_hw']
    s3_meta_GA.spec_hw = best['spec_hw']
    # Stage 4
    s4_meta_GA.diffthresh = best['diffthresh']
    s4_meta_GA.bg_hw = best['bg_hw']
    s4_meta_GA.spec_hw = best['spec_hw']
    # Stage 5
    s5_meta_GA.diffthresh = best['diffthresh']
    s5_meta_GA.bg_hw = best['bg_hw']
    s5_meta_GA.spec_hw = best['spec_hw']


    # Perform parametric sweep
    best_param, best_fitness_value = optimizers.parametric_sweep_S3(objective_funcs.p3thresh, bounds_p3thresh, eventlabel, s3_meta_GA, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

    best['p3thresh'] = best_param[0]

    print("Best parameters: ", best_param[0])
    print("Best fitness: ", best_fitness_value)

    history_fitness_score['p3thresh'] = best_fitness_value

In [ ]:
## STEP 1.0 - Parametric Sweep of "median_thresh" ##

if optimizer == "parametric":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
	s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
	s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
	s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
	s5_meta_GA.fit_method = 'lsq'  


	# Setup Meta / Define Initial Population
	# Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
	s3_meta_GA.bg_hw = best['bg_hw']
	s3_meta_GA.spec_hw = best['spec_hw']
	s3_meta_GA.p3thresh = best['p3thresh']

	# Stage 4
	s4_meta_GA.diffthresh = best['diffthresh']
	s4_meta_GA.bg_hw = best['bg_hw']
	s4_meta_GA.spec_hw = best['spec_hw']
	s4_meta_GA.p3thresh = best['p3thresh']

	# Stage 5
	s5_meta_GA.diffthresh = best['diffthresh']
	s5_meta_GA.bg_hw = best['bg_hw']
	s5_meta_GA.spec_hw = best['spec_hw']
	s5_meta_GA.p3thresh = best['p3thresh']


	# Perform parametric sweep
	best_param, best_fitness_value = optimizers.parametric_sweep_S3(objective_funcs.median_thresh, bounds_median_thresh, eventlabel, s3_meta_GA, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

	best['median_thresh'] = best_param[0]

	print("Best parameters: ", best_param[0])
	print("Best fitness: ", best_fitness_value)

	history_fitness_score['median_thresh'] = best_fitness_value

In [ ]:
## STEP 1.0 - Parametric Sweep of "window_len" ##

if optimizer == "parametric":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
	s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
	s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
	s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
	s5_meta_GA.fit_method = 'lsq'  


	# Setup Meta / Define Initial Population
	# Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
	s3_meta_GA.bg_hw = best['bg_hw']
	s3_meta_GA.spec_hw = best['spec_hw']
	s3_meta_GA.p3thresh = best['p3thresh']

	# Stage 4
	s4_meta_GA.diffthresh = best['diffthresh']
	s4_meta_GA.bg_hw = best['bg_hw']
	s4_meta_GA.spec_hw = best['spec_hw']
	s4_meta_GA.p3thresh = best['p3thresh']

	# Stage 5
	s5_meta_GA.diffthresh = best['diffthresh']
	s5_meta_GA.bg_hw = best['bg_hw']
	s5_meta_GA.spec_hw = best['spec_hw']
	s5_meta_GA.p3thresh = best['p3thresh']


	# Perform parametric sweep for odd numbers
	best_param, best_fitness_value = optimizers.parametric_sweep_odd(objective_funcs.window_len, bounds_window_len, eventlabel, s3_meta_GA, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

	best['window_len'] = best_param[0]

	print("Best parameters: ", best_param[0])
	print("Best fitness: ", best_fitness_value)

	history_fitness_score['window_len'] = best_fitness_value

In [ ]:
## STEP 1.0 - Parametric Sweep of "p5thresh" ##

if optimizer == "parametric":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
	s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
	s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
	s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
	s5_meta_GA.fit_method = 'lsq'  


	# Setup Meta / Define Initial Population
	# Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
	s3_meta_GA.bg_hw = best['bg_hw']
	s3_meta_GA.spec_hw = best['spec_hw']
	s3_meta_GA.p3thresh = best['p3thresh']
	s3_meta_GA.median_thresh = best['median_thresh']
	s3_meta_GA.window_len = best['window_len']

	# Stage 4
	s4_meta_GA.diffthresh = best['diffthresh']
	s4_meta_GA.bg_hw = best['bg_hw']
	s4_meta_GA.spec_hw = best['spec_hw']
	s4_meta_GA.p3thresh = best['p3thresh']
	s4_meta_GA.median_thresh = best['median_thresh']
	s4_meta_GA.window_len = best['window_len']

	# Stage 5
	s5_meta_GA.diffthresh = best['diffthresh']
	s5_meta_GA.bg_hw = best['bg_hw']
	s5_meta_GA.spec_hw = best['spec_hw']
	s5_meta_GA.p3thresh = best['p3thresh']
	s5_meta_GA.median_thresh = best['median_thresh']
	s5_meta_GA.window_len = best['window_len']


	# Perform parametric sweep
	best_param, best_fitness_value = optimizers.parametric_sweep_S3(objective_funcs.p5thresh, bounds_p5thresh, eventlabel, s3_meta_GA, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

	best['p5thresh'] = best_param[0]

	print("Best parameters: ", best_param[0])
	print("Best fitness: ", best_fitness_value)

	history_fitness_score['p5thresh'] = best_fitness_value

In [ ]:
## STEP 1.0 - Parametric Sweep of "p7thresh" ##

if optimizer == "parametric":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
	s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
	s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
	s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
	s5_meta_GA.fit_method = 'lsq'  


	# Setup Meta / Define Initial Population
	# Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
	s3_meta_GA.bg_hw = best['bg_hw']
	s3_meta_GA.spec_hw = best['spec_hw']
	s3_meta_GA.p3thresh = best['p3thresh']
	s3_meta_GA.median_thresh = best['median_thresh']
	s3_meta_GA.window_len = best['window_len']
	s3_meta_GA.p5thresh = best['p5thresh']

	# Stage 4
	s4_meta_GA.diffthresh = best['diffthresh']
	s4_meta_GA.bg_hw = best['bg_hw']
	s4_meta_GA.spec_hw = best['spec_hw']
	s4_meta_GA.p3thresh = best['p3thresh']
	s4_meta_GA.median_thresh = best['median_thresh']
	s4_meta_GA.window_len = best['window_len']
	s4_meta_GA.p5thresh = best['p5thresh']

	# Stage 5
	s5_meta_GA.diffthresh = best['diffthresh']
	s5_meta_GA.bg_hw = best['bg_hw']
	s5_meta_GA.spec_hw = best['spec_hw']
	s5_meta_GA.p3thresh = best['p3thresh']
	s5_meta_GA.median_thresh = best['median_thresh']
	s5_meta_GA.window_len = best['window_len']
	s5_meta_GA.p5thresh = best['p5thresh']


	# Perform parametric sweep
	best_param, best_fitness_value = optimizers.parametric_sweep_S3(objective_funcs.p7thresh, bounds_p7thresh, eventlabel, s3_meta_GA, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

	best['p7thresh'] = best_param[0]

	print("Best parameters: ", best_param[0])
	print("Best fitness: ", best_fitness_value)

	history_fitness_score['p7thresh'] = best_fitness_value

In [ ]:
## STEP 1.0 - Run Stage 3 once and store Stage 3 directory to avoid re-running Stage 3 during Stage 4 optimization (Saves Time)

if optimizer == "parametric":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

	# Setup Meta / Define Initial Population
	# Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
	s3_meta_GA.bg_hw = best['bg_hw']
	s3_meta_GA.spec_hw = best['spec_hw']
	s3_meta_GA.p3thresh = best['p3thresh']
	s3_meta_GA.median_thresh = best['median_thresh']
	s3_meta_GA.p5thresh = best['p5thresh']
	s3_meta_GA.p7thresh = best['p7thresh']
	s3_meta_GA.window_len = best['window_len']

	if __name__ == '__main__':
		s3_spec, s3_meta = s3.reduce(eventlabel, input_meta=s3_meta_GA)
		# s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, input_meta=s4_meta_GA, s3_meta=s3_meta)
		# shutil.rmtree(s3_meta.outputdir)
		# shutil.rmtree(s4_meta.outputdir)

	last_s3_meta_outputdir = s3_meta.outputdir

In [ ]:
## STEP 1.0 - Parametric Sweep of "drift_range" ##

if optimizer == "parametric":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
	s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
	s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
	s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
	s5_meta_GA.fit_method = 'lsq'  


	# Setup Meta / Define Initial Population
	# Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
	s3_meta_GA.bg_hw = best['bg_hw']
	s3_meta_GA.spec_hw = best['spec_hw']
	s3_meta_GA.p3thresh = best['p3thresh']
	s3_meta_GA.median_thresh = best['median_thresh']
	s3_meta_GA.window_len = best['window_len']
	s3_meta_GA.p5thresh = best['p5thresh']
	s3_meta_GA.p7thresh = best['p7thresh']

	# Stage 4
	s4_meta_GA.diffthresh = best['diffthresh']
	s4_meta_GA.bg_hw = best['bg_hw']
	s4_meta_GA.spec_hw = best['spec_hw']
	s4_meta_GA.p3thresh = best['p3thresh']
	s4_meta_GA.median_thresh = best['median_thresh']
	s4_meta_GA.window_len = best['window_len']
	s4_meta_GA.p5thresh = best['p5thresh']
	s4_meta_GA.p7thresh = best['p7thresh']

	# Stage 4
	s5_meta_GA.diffthresh = best['diffthresh']
	s5_meta_GA.bg_hw = best['bg_hw']
	s5_meta_GA.spec_hw = best['spec_hw']
	s5_meta_GA.p3thresh = best['p3thresh']
	s5_meta_GA.median_thresh = best['median_thresh']
	s5_meta_GA.window_len = best['window_len']
	s5_meta_GA.p5thresh = best['p5thresh']
	s5_meta_GA.p7thresh = best['p7thresh']


	# Perform parametric sweep
	best_param, best_fitness_value = optimizers.parametric_sweep_S4(objective_funcs.drift_range, bounds_highpassWidth, eventlabel, last_s3_meta_outputdir, s3_meta, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

	best['drift_range'] = best_param[0]

	print("Best parameters: ", best_param[0])
	print("Best fitness: ", best_fitness_value)

	history_fitness_score['drift_range'] = best_fitness_value

In [ ]:
## STEP 1.0 - Parametric Sweep of "highpassWidth" ##

if optimizer == "parametric":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
	s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
	s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
	s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
	s5_meta_GA.fit_method = 'lsq'  


	# Setup Meta / Define Initial Population
	# Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
	s3_meta_GA.bg_hw = best['bg_hw']
	s3_meta_GA.spec_hw = best['spec_hw']
	s3_meta_GA.p3thresh = best['p3thresh']
	s3_meta_GA.median_thresh = best['median_thresh']
	s3_meta_GA.window_len = best['window_len']
	s3_meta_GA.p5thresh = best['p5thresh']
	s3_meta_GA.p7thresh = best['p7thresh']

	# Stage 4
	s4_meta_GA.diffthresh = best['diffthresh']
	s4_meta_GA.bg_hw = best['bg_hw']
	s4_meta_GA.spec_hw = best['spec_hw']
	s4_meta_GA.p3thresh = best['p3thresh']
	s4_meta_GA.median_thresh = best['median_thresh']
	s4_meta_GA.window_len = best['window_len']
	s4_meta_GA.p5thresh = best['p5thresh']
	s4_meta_GA.p7thresh = best['p7thresh']
	s4_meta_GA.drift_range = best['drift_range']

	# Stage 4
	s5_meta_GA.diffthresh = best['diffthresh']
	s5_meta_GA.bg_hw = best['bg_hw']
	s5_meta_GA.spec_hw = best['spec_hw']
	s5_meta_GA.p3thresh = best['p3thresh']
	s5_meta_GA.median_thresh = best['median_thresh']
	s5_meta_GA.window_len = best['window_len']
	s5_meta_GA.p5thresh = best['p5thresh']
	s5_meta_GA.p7thresh = best['p7thresh']
	s5_meta_GA.drift_range = best['drift_range']


	# Perform parametric sweep
	best_param, best_fitness_value = optimizers.parametric_sweep_S4(objective_funcs.highpassWidth, bounds_highpassWidth, eventlabel, last_s3_meta_outputdir, s3_meta, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

	best['highpassWidth'] = best_param[0]

	print("Best parameters: ", best_param[0])
	print("Best fitness: ", best_fitness_value)

	history_fitness_score['highpassWidth'] = best_fitness_value

In [ ]:
## STEP 1.0 - Parametric Sweep of "sigma" ##

if optimizer == "parametric":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
	s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
	s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
	s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
	s5_meta_GA.fit_method = 'lsq'  


	# Setup Meta / Define Initial Population
	# Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
	s3_meta_GA.bg_hw = best['bg_hw']
	s3_meta_GA.spec_hw = best['spec_hw']
	s3_meta_GA.p3thresh = best['p3thresh']
	s3_meta_GA.median_thresh = best['median_thresh']
	s3_meta_GA.window_len = best['window_len']
	s3_meta_GA.p5thresh = best['p5thresh']
	s3_meta_GA.p7thresh = best['p7thresh']

	# Stage 4
	s4_meta_GA.diffthresh = best['diffthresh']
	s4_meta_GA.bg_hw = best['bg_hw']
	s4_meta_GA.spec_hw = best['spec_hw']
	s4_meta_GA.p3thresh = best['p3thresh']
	s4_meta_GA.median_thresh = best['median_thresh']
	s4_meta_GA.window_len = best['window_len']
	s4_meta_GA.p5thresh = best['p5thresh']
	s4_meta_GA.p7thresh = best['p7thresh']
	s4_meta_GA.drift_range = best['drift_range']
	s4_meta_GA.highpassWidth = best['highpassWidth']

	# Stage 5
	s5_meta_GA.diffthresh = best['diffthresh']
	s5_meta_GA.bg_hw = best['bg_hw']
	s5_meta_GA.spec_hw = best['spec_hw']
	s5_meta_GA.p3thresh = best['p3thresh']
	s5_meta_GA.median_thresh = best['median_thresh']
	s5_meta_GA.window_len = best['window_len']
	s5_meta_GA.p5thresh = best['p5thresh']
	s5_meta_GA.p7thresh = best['p7thresh']
	s5_meta_GA.drift_range = best['drift_range']
	s5_meta_GA.highpassWidth = best['highpassWidth']
	

	# Perform parametric sweep
	best_param, best_fitness_value = optimizers.parametric_sweep_S4(objective_funcs.sigma, bounds_sigma, eventlabel, last_s3_meta_outputdir, s3_meta, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

	best['sigma'] = best_param[0]

	print("Best parameters: ", best_param[0])
	print("Best fitness: ", best_fitness_value)

	history_fitness_score['sigma'] = best_fitness_value

In [ ]:
## STEP 1.0 - Parametric Sweep of "box_width" ##

if optimizer == "parametric":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
	s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
	s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
	s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
	s5_meta_GA.fit_method = 'lsq'  


	# Setup Meta / Define Initial Population
	# Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
	s3_meta_GA.bg_hw = best['bg_hw']
	s3_meta_GA.spec_hw = best['spec_hw']
	s3_meta_GA.p3thresh = best['p3thresh']
	s3_meta_GA.median_thresh = best['median_thresh']
	s3_meta_GA.window_len = best['window_len']
	s3_meta_GA.p5thresh = best['p5thresh']
	s3_meta_GA.p7thresh = best['p7thresh']

	# Stage 4
	s4_meta_GA.diffthresh = best['diffthresh']
	s4_meta_GA.bg_hw = best['bg_hw']
	s4_meta_GA.spec_hw = best['spec_hw']
	s4_meta_GA.p3thresh = best['p3thresh']
	s4_meta_GA.median_thresh = best['median_thresh']
	s4_meta_GA.window_len = best['window_len']
	s4_meta_GA.p5thresh = best['p5thresh']
	s4_meta_GA.p7thresh = best['p7thresh']
	s4_meta_GA.drift_range = best['drift_range']
	s4_meta_GA.highpassWidth = best['highpassWidth']
	s4_meta_GA.sigma = best['sigma']

	# Stage 5
	s5_meta_GA.diffthresh = best['diffthresh']
	s5_meta_GA.bg_hw = best['bg_hw']
	s5_meta_GA.spec_hw = best['spec_hw']
	s5_meta_GA.p3thresh = best['p3thresh']
	s5_meta_GA.median_thresh = best['median_thresh']
	s5_meta_GA.window_len = best['window_len']
	s5_meta_GA.p5thresh = best['p5thresh']
	s5_meta_GA.p7thresh = best['p7thresh']
	s5_meta_GA.drift_range = best['drift_range']
	s5_meta_GA.highpassWidth = best['highpassWidth']
	s5_meta_GA.sigma = best['sigma']


	# Perform parametric sweep
	best_param, best_fitness_value = optimizers.parametric_sweep_S4(objective_funcs.box_width, bounds_box_width, eventlabel, last_s3_meta_outputdir, s3_meta, s4_meta_GA, s5_meta_GA, params, scaling_MAD_white, scaling_MAD_spec, scaling_chi2red)

	best['box_width'] = best_param[0]

	print("Best parameters: ", best_param[0])
	print("Best fitness: ", best_fitness_value)

	history_fitness_score['box_width'] = best_fitness_value

In [ ]:
## STEP 1.1.1 - Genetic Algorithm - STEP 1 - Calculations ## -- Working, but best individuals not integers

if optimizer == "genetic":

    print("Running genetic algorithm.")

    ## Setup Meta ##
    s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
    s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
    s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Load Optimized ECF Values
	# Stage 3
    s3_meta_GA.diffthresh = best['diffthresh']
    s3_meta_GA.bg_hw = best['bg_hw']
    s3_meta_GA.spec_hw = best['spec_hw']

	# Stage 4
    s4_meta_GA.diffthresh = best['diffthresh']
    s4_meta_GA.bg_hw = best['bg_hw']
    s4_meta_GA.spec_hw = best['spec_hw']

	# Stage 5
    s5_meta_GA.diffthresh = best['diffthresh']
    s5_meta_GA.bg_hw = best['bg_hw']
    s5_meta_GA.spec_hw = best['spec_hw']

    # Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
    s3_meta_GA.xwindow = ev.xwindow
    s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
    s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
    s5_meta_GA.fit_method = 'lsq'     

    # Mutation Rate
    mutation_rate = 1/len(max_bounds)

    # Define Initial Population
    initialPop_Step_1 = [
                    # s3_meta_GA.diffthresh,
                    s3_meta_GA.p3thresh,
                    s3_meta_GA.median_thresh,
                    s3_meta_GA.window_len,
                    s3_meta_GA.p5thresh,
                    s3_meta_GA.p7thresh]
    

    # Objective Function
    def objective_function_GA_Step_1(x):

        try:

            # Ensure window_len is an odd number and is within bounds
            window_len = int(round(x[2]))  # Round to nearest integer
            if window_len % 2 == 0:  # Check if the number is even
                if window_len < bounds_window_len[1]:  # If it's not exceeding the max bound
                    window_len += 1  # Make it odd by adding 1
                else:  # If it was already at max bound
                    window_len -= 1  # Make it odd by subtracting 1
                    
            # Ensure it's within bounds, even after making it odd
            window_len = max(min(window_len, bounds_window_len[1]), bounds_window_len[0])

            # Define Variables to be optimized
            # Stage 3
            # s3_meta_GA.diffthresh = int(x[0])
            s3_meta_GA.p3thresh = int(x[0])
            s3_meta_GA.median_thresh = int(x[1])
            s3_meta_GA.window_len = int(window_len)
            s3_meta_GA.p5thresh = int(x[3])
            s3_meta_GA.p7thresh = int(x[4])
            # Stage 4
            # s4_meta_GA.diffthresh = int(x[0])
            s4_meta_GA.p3thresh = int(x[0])
            s4_meta_GA.median_thresh = int(x[1])
            s4_meta_GA.window_len = int(window_len)
            s4_meta_GA.p5thresh = int(x[3])
            s4_meta_GA.p7thresh = int(x[4])
            # Stage 5
            # s5_meta_GA.diffthresh = int(x[0])
            s5_meta_GA.p3thresh = int(x[0])
            s5_meta_GA.median_thresh = int(x[1])
            s5_meta_GA.window_len = int(window_len)
            s5_meta_GA.p5thresh = int(x[3])
            s5_meta_GA.p7thresh = int(x[4])

            # Run Eureka!
            # if __name__ == '__main__':
            s3_spec, s3_meta = s3.reduce(eventlabel, input_meta=s3_meta_GA)
            s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, input_meta=s4_meta_GA, s3_meta=s3_meta)
            s5_meta = s5.fitlc(eventlabel, params, input_meta=s5_meta_GA, s4_meta=s4_meta)
            
            fitness_value = ( scaling_chi2red * s5_meta.chi2red ) + (scaling_MAD_spec * s4_meta.mad_s4 ) + ( scaling_MAD_white  * ( sum(s4_meta.mad_s4_binned)  /  len(s4_meta.mad_s4_binned) ) )
        
            shutil.rmtree(s3_meta.outputdir)
            shutil.rmtree(s4_meta.outputdir)
            shutil.rmtree(s5_meta.outputdir)
            
            return fitness_value
        
        except Exception:  # catch any exception

            return np.nan  # return nan if there was an error

    
    def fitness_Step_1(population):
        fitness_values = np.array([objective_function_GA_Step_1(individual) for individual in population])
        valid_indices = np.isfinite(fitness_values)  # change from np.logical_not(np.isnan(fitness_values)) to handle inf values as well
        return fitness_values[valid_indices], population[valid_indices]

In [ ]:
## STEP 1.1.1 - Genetic Algorithm - STEP 1 - Run Genetic Algorithm ##

if optimizer == "genetic":
    
    best_individuals_Step_1, best_fitness_values_Step_1 = optimizers.genetic_algorithm(population_size_Step_1, generations_Step_1, 
                                                                                       min_bounds_Step_1, max_bounds_Step_1, 
                                                                                       initialPop_Step_1, mutation_rate, 
                                                                                       fitness_Step_1, eventlabel,
                                                                                       s3_meta_GA, s4_meta_GA, s5_meta_GA, params,
                                                                                       scaling_MAD_white, scaling_MAD_spec, scaling_chi2red,
                                                                                       target_fitness)

In [ ]:
## STEP 1.1.1 - Genetic Algorithm - STEP 1 - Plot Fitness ##

if optimizer == "genetic":

    optimizers.plot_fitness_scores(best_fitness_values_Step_1)

    history_fitness_score['GA_Step1'] = best_fitness_values_Step_1

In [ ]:
## STEP 1.1.1 - Genetic Algorithm - STEP 1 - Select Best Individual from GA-based Optimization and Save Values ##

if optimizer == "genetic":

    best_individual = optimizers.select_best_individual(best_individuals_Step_1, best_fitness_values_Step_1)
    print("Best individual:", best_individual)

    # best['diffthresh'] = int(best_individual[0])
    best['p3thresh'] = int(best_individual[0])
    best['median_thresh'] = int(best_individual[1])
    best['window_len'] = int(best_individual[2])
    best['p5thresh'] = int(best_individual[3])
    best['p7thresh'] = int(best_individual[4])

    print(best)

In [ ]:
## STEP 1.1.1.1 - Run Stage 3 once and store the Stage 3 directory to avoid re-running Stage 3 during Stage 4 optimization (Saves Time)

if optimizer == "genetic":

	## Setup Meta ##
	s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)

	# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
	s3_meta_GA.xwindow = ev.xwindow
	s3_meta_GA.ywindow = ev.ywindow

	# Setup Meta / Define Initial Population
	# Stage 3
	s3_meta_GA.diffthresh = best['diffthresh']
	s3_meta_GA.bg_hw = best['bg_hw']
	s3_meta_GA.spec_hw = best['spec_hw']
	s3_meta_GA.p3thresh = best['p3thresh']
	s3_meta_GA.median_thresh = best['median_thresh']
	s3_meta_GA.p5thresh = best['p5thresh']
	s3_meta_GA.p7thresh = best['p7thresh']
	s3_meta_GA.window_len = best['window_len']

	if __name__ == '__main__':
		s3_spec, s3_meta = s3.reduce(eventlabel, input_meta=s3_meta_GA)
		# s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, input_meta=s4_meta_GA, s3_meta=s3_meta)
		# shutil.rmtree(s3_meta.outputdir)
		# shutil.rmtree(s4_meta.outputdir)

	last_s3_meta_outputdir = s3_meta.outputdir

In [ ]:
## STEP 1.1.2 - Genetic Algorithm - STEP 2 - Calculations ## -- Working, but best individuals not integers

if optimizer == "genetic":

    print("Running genetic algorithm.")

    ## Setup Meta ##
    # s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
    s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
    s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

	# Load Optimized ECF Values
	# # Stage 3
    # s3_meta_GA.diffthresh = best['diffthresh']
    # s3_meta_GA.bg_hw = best['bg_hw']
    # s3_meta_GA.spec_hw = best['spec_hw']
    # s3_meta_GA.p3thresh = best['p3thresh']
    # s3_meta_GA.median_thresh = best['median_thresh']
    # s3_meta_GA.window_len = best['window_len']
    # s3_meta_GA.p5thresh = best['p5thresh']
    # s3_meta_GA.p7thresh = best['p7thresh']

	# Stage 4
    s4_meta_GA.diffthresh = best['diffthresh']
    s4_meta_GA.bg_hw = best['bg_hw']
    s4_meta_GA.spec_hw = best['spec_hw']
    s4_meta_GA.p3thresh = best['p3thresh']
    s4_meta_GA.median_thresh = best['median_thresh']
    s4_meta_GA.window_len = best['window_len']
    s4_meta_GA.p5thresh = best['p5thresh']
    s4_meta_GA.p7thresh = best['p7thresh']

	# Stage 5
    s5_meta_GA.diffthresh = best['diffthresh']
    s5_meta_GA.bg_hw = best['bg_hw']
    s5_meta_GA.spec_hw = best['spec_hw']
    s5_meta_GA.p3thresh = best['p3thresh']
    s5_meta_GA.median_thresh = best['median_thresh']
    s5_meta_GA.window_len = best['window_len']
    s5_meta_GA.p5thresh = best['p5thresh']
    s5_meta_GA.p7thresh = best['p7thresh']
    

    # Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
    s3_meta_GA.xwindow = ev.xwindow
    s3_meta_GA.ywindow = ev.ywindow

    # Set Up Manual Clipping
    s5_meta_GA.manual_clip = manual_clip_lists

    # Set Fit Method
    s5_meta_GA.fit_method = 'lsq'     

    # Mutation Rate
    mutation_rate = 1/len(max_bounds)

    # Define Initial Population
    initialPop_Step_2 = [s4_meta_GA.drift_range,
                    s4_meta_GA.highpassWidth,
                    s4_meta_GA.sigma,
                    s4_meta_GA.box_width]
    

    # Objective Function
    def objective_function_GA_Step_2(x):

        try:

            # Define Variables to be optimized
            # Stage 4
            s4_meta_GA.drift_range = int(x[0])
            s4_meta_GA.highpassWidth = int(x[1])
            s4_meta_GA.sigma = int(x[2])
            s4_meta_GA.box_width = int(x[3])
            # Stage 5
            s5_meta_GA.drift_range = int(x[0])
            s5_meta_GA.highpassWidth = int(x[1])
            s5_meta_GA.sigma = int(x[2])
            s5_meta_GA.box_width = int(x[3])

            # Run Eureka!
            # if __name__ == '__main__':
            s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, input_meta=s4_meta_GA, s3_meta=s3_meta)
            s5_meta = s5.fitlc(eventlabel, params, input_meta=s5_meta_GA, s4_meta=s4_meta)
            
            fitness_value = ( scaling_chi2red * s5_meta.chi2red ) + (scaling_MAD_spec * s4_meta.mad_s4 ) + ( scaling_MAD_white  * ( sum(s4_meta.mad_s4_binned)  /  len(s4_meta.mad_s4_binned) ) )
        
            shutil.rmtree(s4_meta.outputdir)
            shutil.rmtree(s5_meta.outputdir)
            
            return fitness_value

        except Exception:  # catch any exception
            return np.nan  # return nan if there was an error

    
    def fitness_Step_2(population):
        fitness_values = np.array([objective_function_GA_Step_2(individual) for individual in population])
        valid_indices = np.isfinite(fitness_values)  # change from np.logical_not(np.isnan(fitness_values)) to handle inf values as well
        return fitness_values[valid_indices], population[valid_indices]

In [ ]:
## STEP 1.1.2 - Genetic Algorithm - STEP 2 - Run Genetic Algorithm ##

if optimizer == "genetic":
    
    s4_meta_GA.inputdir = last_s3_meta_outputdir

    best_individuals_Step_2, best_fitness_values_Step_2 = optimizers.genetic_algorithm(population_size_Step_2, generations_Step_2, 
                                                                                       min_bounds_Step_2, max_bounds_Step_2, 
                                                                                       initialPop_Step_2, mutation_rate, 
                                                                                       fitness_Step_2, eventlabel,
                                                                                       s3_meta, s4_meta_GA, s5_meta_GA, params,
                                                                                       scaling_MAD_white, scaling_MAD_spec, scaling_chi2red,
                                                                                       target_fitness)
    
    history_fitness_score['GA_Step2'] = best_fitness_values_Step_2

In [ ]:
## STEP 1.1.2 - Genetic Algorithm - STEP 2 - Plot Fitness ##

if optimizer == "genetic":

    optimizers.plot_fitness_scores(best_fitness_values_Step_2)

In [ ]:
## STEP 1.1.2 - Genetic Algorithm - STEP 2 - Select Best Individual from GA-based Optimization and Save Values ##

if optimizer == "genetic":

    best_individual = optimizers.select_best_individual(best_individuals_Step_2, best_fitness_values_Step_2)
    print("Best individual:", best_individual)

    best['drift_range'] = int(best_individual[0])
    best['highpassWidth'] = int(best_individual[1])
    best['sigma'] = int(best_individual[2])
    best['box_width'] = int(best_individual[3])

    print(best)

In [ ]:
## STEP 1.2 - Save Best ECF Values ##

# Save Best Window Positions
ev.xwindow[0] = best['xwindow_LB']
ev.xwindow[1] = best['xwindow_UB']

# Save Best Window Positions
ev.ywindow[0] = best['ywindow_LB']
ev.ywindow[1] = best['ywindow_UB']

# Setup Meta
s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

s3_meta_GA.xwindow = ev.xwindow
s3_meta_GA.ywindow = ev.ywindow

# Stage 3
s3_meta_GA.diffthresh = best['diffthresh']
s3_meta_GA.bg_hw = best['bg_hw']
s3_meta_GA.spec_hw = best['spec_hw']
s3_meta_GA.p3thresh = best['p3thresh']
s3_meta_GA.median_thresh = best['median_thresh']
s3_meta_GA.window_len = best['window_len']
s3_meta_GA.p5thresh = best['p5thresh']
s3_meta_GA.p7thresh = best['p7thresh']
# Stage 4
s4_meta_GA.diffthresh = best['diffthresh']
s4_meta_GA.bg_hw = best['bg_hw']
s4_meta_GA.spec_hw = best['spec_hw']
s4_meta_GA.p3thresh = best['p3thresh']
s4_meta_GA.median_thresh = best['median_thresh']
s4_meta_GA.window_len = best['window_len']
s4_meta_GA.p5thresh = best['p5thresh']
s4_meta_GA.p7thresh = best['p7thresh']
s4_meta_GA.drift_range = best['drift_range']
s4_meta_GA.highpassWidth = best['highpassWidth']
s4_meta_GA.sigma = best['sigma']
s4_meta_GA.box_width = best['box_width']
# Stage 5
s5_meta_GA.diffthresh = best['diffthresh']
s5_meta_GA.bg_hw = best['bg_hw']
s5_meta_GA.spec_hw = best['spec_hw']
s5_meta_GA.p3thresh = best['p3thresh']
s5_meta_GA.median_thresh = best['median_thresh']
s5_meta_GA.window_len = best['window_len']
s5_meta_GA.p5thresh = best['p5thresh']
s5_meta_GA.p7thresh = best['p7thresh']
s5_meta_GA.drift_range = best['drift_range']
s5_meta_GA.highpassWidth = best['highpassWidth']
s5_meta_GA.sigma = best['sigma']
s5_meta_GA.box_width = best['box_width']


In [ ]:
## STEP 1.2 - Save Best EPF Values ##

# Fitted Params
best['rp'] = params.rp.value
best['per'] = params.per.value
best['t0'] = params.t0.value
best['inc'] = params.inc.value
best['a'] = params.a.value
best['ecc'] = params.ecc.value
best['omega'] = params.w.value
best['u1'] = params.u1.value
best['u2'] = params.u2.value
best['c0'] = params.c0.value
best['c1'] = params.c1.value
best['h0'] = params.h0.value
best['h1'] = params.h1.value
best['h3'] = params.h3.value
best['h5'] = params.h5.value
best['h6'] = params.h6.value
best['scatter_mult'] = params.scatter_mult.value

print(f"ev.xwindow = {ev.xwindow}")
print(f"Optimized ECF and EPF Inputs for Stages 3, 4, 5 : {best}")

In [ ]:
# # ## DEBUG - Previous Best ##

# # # Use pickle.load to load the "best" dictionary
# # # with open(outputdir_optimization + "best_inputs.pkl", "rb") as f:
# with open("/home/DataAnalysis/HST/HD86226c/Optimized/Visit36-39/Optimized_2024-09-09_HD86226c_run1/" + "best_inputs.pkl", "rb") as f:
#     best = pickle.load(f)


In [ ]:
## STEP 1.3 - Test the Optimized White Light Curve  - Stages 3 & 4 ##

## Setup Meta ##
s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)

# Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
# s3_meta_GA.xwindow = ev.xwindow
# s3_meta_GA.ywindow = ev.ywindow

s3_meta_GA.xwindow = [best['xwindow_LB'], best['xwindow_UB']]
# s3_meta_GA.ywindow = [best['ywindow_LB'], best['ywindow_UB']]
if "ywindow_LB" in best:
    s3_meta_GA.ywindow = [best['ywindow_LB'], best['ywindow_UB']]
else:
    best['ywindow_LB'] = ev.ywindow[0]
    best['ywindow_UB'] = ev.ywindow[1]
    s3_meta_GA.ywindow = [best['ywindow_LB'], best['ywindow_UB']]

# Manual Clipping
# s5_meta_GA.manual_clip = manual_clip_lists

# Fit Method
s5_meta_GA.fit_method = 'emcee'

# Hide Plots
s5_meta_GA.hide_plots = False

# Overwrite default meta inputs with optimized Stage 3 and Stage 4 inputs
# Stage 3
s3_meta_GA.diffthresh = best['diffthresh']
s3_meta_GA.bg_hw = best['bg_hw']
s3_meta_GA.spec_hw = best['spec_hw']
s3_meta_GA.p3thresh = best['p3thresh']
s3_meta_GA.median_thresh = best['median_thresh']
s3_meta_GA.window_len = best['window_len']
s3_meta_GA.p5thresh = best['p5thresh']
s3_meta_GA.p7thresh = best['p7thresh']
# Stage 4
s4_meta_GA.diffthresh = best['diffthresh']
s4_meta_GA.bg_hw = best['bg_hw']
s4_meta_GA.spec_hw = best['spec_hw']
s4_meta_GA.p3thresh = best['p3thresh']
s4_meta_GA.median_thresh = best['median_thresh']
s4_meta_GA.window_len = best['window_len']
s4_meta_GA.p5thresh = best['p5thresh']
s4_meta_GA.p7thresh = best['p7thresh']
s4_meta_GA.drift_range = best['drift_range']
s4_meta_GA.highpassWidth = best['highpassWidth']
s4_meta_GA.sigma = best['sigma']
s4_meta_GA.box_width = best['box_width']
# Stage 5
s5_meta_GA.diffthresh = best['diffthresh']
s5_meta_GA.bg_hw = best['bg_hw']
s5_meta_GA.spec_hw = best['spec_hw']
s5_meta_GA.p3thresh = best['p3thresh']
s5_meta_GA.median_thresh = best['median_thresh']
s5_meta_GA.window_len = best['window_len']
s5_meta_GA.p5thresh = best['p5thresh']
s5_meta_GA.p7thresh = best['p7thresh']
s5_meta_GA.drift_range = best['drift_range']
s5_meta_GA.highpassWidth = best['highpassWidth']
s5_meta_GA.sigma = best['sigma']
s5_meta_GA.box_width = best['box_width']

if enable_exotic_ld is True:

    # Retrieve Values for Exotic-ld
    s4_meta_GA.teff = planet_data['Teff']
    s4_meta_GA.logg = planet_data['stellar_gravity']
    s4_meta_GA.metallicity = planet_data['Fe/H']

    s5_meta_GA.teff = planet_data['Teff']
    s5_meta_GA.logg = planet_data['stellar_gravity']
    s5_meta_GA.metallicity = planet_data['Fe/H']

    # Turn on compute_ld
    s4_meta_GA.compute_ld = True

    # Turn on compute white
    s4_meta_GA.compute_white = True

    # Specify 1D or 3D Grid Model
    s4_meta_GA.exotic_ld_grid = '3D'

    # Path for exotic-ld ancillary files 
    s4_meta_GA.exotic_ld_direc = exotic_ld_direc

    # Path for exotic-ld throughput file 
    s4_meta_GA.exotic_ld_file = exotic_ld_file

    # Turn on use_generate_ld and enter paths for ld files
    s5_meta_GA.use_generate_ld = use_generate_ld

    # Path for ld file (white) 
    s5_meta_GA.ld_file = ld_file

    # Path for ld file (white) 
    s5_meta_GA.ld_file_white = ld_file_white


directory = s4_meta_GA.topdir
# directory = last_outputdir_S4
if not os.path.exists(directory):
    os.makedirs(directory)


# Run Test
if __name__ == '__main__':
    s3_spec, s3_meta = s3.reduce(eventlabel, input_meta=s3_meta_GA)
    s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, input_meta=s4_meta_GA, s3_meta=s3_meta)

In [ ]:

# directory = s4_meta_GA.topdir
# # directory = last_outputdir_S4
# if not os.path.exists(directory):
#     os.makedirs(directory)


# # Run Test
# if __name__ == '__main__':
#     s3_spec, s3_meta = s3.reduce(eventlabel, input_meta=s3_meta_GA)
#     s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, input_meta=s4_meta_GA, s3_meta=s3_meta)

In [ ]:
## STEP 1.3 - Test the Optimized White Light Curve  - Stage 5 ##

# Run Test
if __name__ == '__main__':
    # try:

    #     # Call the Generate EPF function
    #     generate_epf.fixed(best, file_path=os.path.join(directory, s5_epffile)) # All Fixed

    #     # Load params object from EPF
    #     params = readEPF.Parameters(s5_meta_GA.folder, s5_meta_GA.fit_par)

    #     # Run Test
    #     s5_meta = s5.fitlc(eventlabel, params, input_meta=s5_meta_GA, s4_meta=s4_meta)

    # except:

    #     print(f"Error encountered using generate_epf.fixed, likely due to a lack of free parameters"
    #           f"causing a conflict. \n Attempting to use generate_epf.Stage_5.")

    # Call the Generate EPF function
    generate_epf.Stage_5(planet_data, file_path=os.path.join(directory, s5_epffile))

    # Load params object from EPF
    params = readEPF.Parameters(s5_meta_GA.folder, s5_meta_GA.fit_par)

    # Run Test
    s5_meta = s5.fitlc(eventlabel, params, input_meta=s5_meta_GA, s4_meta=s4_meta)


In [ ]:
# ## DEBUG -- Checkpoint for manual LC fitting   

# # Load params object from EPF
# params = readEPF.Parameters(s5_meta_GA.folder, s5_meta_GA.fit_par)

# manual_clip_lists = [[0, 11], [26, 28], [52, 54], [78, 80]] # Standard 

# s5_meta_GA.manual_clip = manual_clip_lists

# s5_meta_GA.fit_method = 'lsq'

# s5_meta = s5.fitlc(eventlabel, params, input_meta=s5_meta_GA, s4_meta=s4_meta)

In [ ]:
# ## STEP 1.3 - Save optimization metrics

# # Save Metrics
# history_MAD_white['Final_Run'] = (sum(s4_meta.mad_s4_binned) / len(s4_meta.mad_s4_binned))
# history_MAD_spec['Final_Run'] = s4_meta.mad_s4
# history_MAD_chi2red['Final_Run'] = s5_meta.chi2red

# best_fitness_value_ = (
#     scaling_chi2red * s5_meta.chi2red +
#     scaling_MAD_spec * s4_meta.mad_s4 +
#     scaling_MAD_white *
#     (sum(s4_meta.mad_s4_binned) / len(s4_meta.mad_s4_binned))
# )

# history_fitness_score['Final_Run'] = best_fitness_value

In [ ]:
# ## STEP 1.3 - Plot of optimization history

# # Extracting keys and values

# keys = list(history_fitness_score.keys())[3:]  # Omit first 3 points
# values = list(history_fitness_score.values())[3:] # Omit first 3 points

# # Creating the plot
# plt.figure(figsize=(10, 6))
# plt.plot(keys, values, marker='o')
# plt.xticks(rotation=45, ha='right')
# plt.ylabel('Fitness Score')
# plt.title('History of Fitness Scores')
# plt.grid(True)
# plt.tight_layout()

# # Show the plot
# plt.show()

In [ ]:
## STEP 1.3 - Save fitted params

best['rp'] = params.rp.value
best['per'] = params.per.value
best['t0'] = params.t0.value
best['inc'] = params.inc.value
best['a'] = params.a.value
best['ecc'] = params.ecc.value
best['omega'] = params.w.value
best['u1'] = params.u1.value
best['u2'] = params.u2.value
best['c0'] = params.c0.value
best['c1'] = params.c1.value
best['h0'] = params.h0.value
best['h1'] = params.h1.value
best['h3'] = params.h3.value
best['h5'] = params.h5.value
best['h6'] = params.h6.value
# best['xpos'] = params.xpos.value
# best['ypos'] = params.ypos.value
best['scatter_mult'] = params.scatter_mult.value
print(f"\n Best = {best} \n")

# Call the Generate EPF function
generate_epf.Stage_6(best, planet_data, file_path=os.path.join(directory, s5_epffile))


## STEP 1.3 - Create Run String for Output Path

def create_run_string(event_label, directory_path):
    # Get today's date in YYYY-MM-DD format
    today_date = datetime.now().strftime("%Y-%m-%d")

    # Initialize the run number to 1
    run_number = 1

    # Check if the directory exists
    if os.path.exists(directory_path):
        # List all files in the directory and find the highest run number
        for file in os.listdir(directory_path):
            if file.startswith(f"Optimized_{today_date}_{event_label}_run"):
                try:
                    current_run_number = int(file.split('_')[-1][3:])
                    run_number = max(run_number, current_run_number + 1)
                except ValueError:
                    continue

    # Create the formatted string
    return f"Optimized_{today_date}_{eventlabel}_run{run_number}"

# Create Run String
run_string = create_run_string(eventlabel, outputdir_optimization)

# Update outputdir to run-specific folder
outputdir_optimization = outputdir_optimization + run_string + "/"


## STEP 1.3 - Save best S3-S5 ECF and EPF values ##

# Use pickle.dump to save the dictionary
if not os.path.exists(outputdir_optimization):
    os.makedirs(outputdir_optimization)

with open(outputdir_optimization + "best_inputs.pkl", "wb") as f:
    pickle.dump(best, f)


## STEP 1.3 - Set save directory for optimized white light curve ##

best_white = best

print(f"best = {best_white}")
print(f"s5_meta.chi2red = {s5_meta.chi2red}")

outputdir_WLC_S3 = s3_meta.outputdir
outputdir_WLC_S4 = s4_meta.outputdir
outputdir_WLC_S5 = s5_meta.outputdir

# Save optimized WLC data for reference
s3_meta_best_WLC = s3_meta
s4_meta_best_WLC = s4_meta
s5_meta_best_WLC = s5_meta


## STEP 1.3 - Copy the optimizer_inputs.txt file used to the optimization output folder ##

def copy_and_rename_file(src, dest_dir, dest_filename):
    # Check if destination directory exists; if not, create it
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Copy and rename the file
    shutil.copy(src, os.path.join(dest_dir, dest_filename))

if __name__ == '__main__':
    source_file = 'optimizer_inputs.txt'
    destination_directory = outputdir_optimization
    destination_filename = 'optimizer_inputs_copy.txt'

    copy_and_rename_file(source_file, destination_directory, destination_filename)
    print(f"'{source_file}' has been copied to '{destination_directory}/{destination_filename}'")

    
## STEP 1.3 - Create output file for optimization results ##

output_file = outputdir_optimization + "optimization_results.txt"

with open(output_file, 'w') as f:
    
    f.write("## OPTIMIZATION RESULTS ##\n\n")

    # Write Dictionary Contents
    f.write("## Optimized ECF & EPF Inputs ##\n")
    for key, value in best.items():
        f.write(f"{key}: {value}\n")

    f.write("\n")

    # Write Optimization Results
    f.write("## Optimization Metrics ##\n")
    f.write(f"MAD_WLC = {sum(s4_meta_best_WLC.mad_s4_binned) / len(s4_meta_best_WLC.mad_s4_binned)}\n")
    f.write(f"MAD_Spec = {s4_meta_best_WLC.mad_s4}\n")
    f.write(f"chi2red_WLC = {s5_meta_best_WLC.chi2red}\n\n")

    f.write("## Manual Clip Lists ##\n\n")
    f.write(f"manual_clip = {manual_clip_lists}\n")

    # f.write(chi2red_nspecchan)

print(f"Optimization results saved to {output_file}")


## STEP 1.3 - Save results for optimized white light curve ##

def copy_files_and_subfolders_to_new_directory(src_dirs, dest_dir):
    # Ensure the destination directory exists
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Iterate over the source directories and copy each file and folder
    for src_dir, folder_name in src_dirs:
        if os.path.exists(src_dir):
            dest_subdir = os.path.join(dest_dir, folder_name)
            if not os.path.exists(dest_subdir):
                os.makedirs(dest_subdir)
            shutil.copytree(src_dir, dest_subdir, dirs_exist_ok=True)

# Assuming these are full path directories
directories = [
    (outputdir_WLC_S3, "S3"),
    (outputdir_WLC_S4, "S4"),
    (outputdir_WLC_S5, "S5")
]

dest_directory_white = outputdir_optimization + "White"

# Copy files and subfolders
copy_files_and_subfolders_to_new_directory(directories, dest_directory_white)

In [ ]:
# TO DO: Automate string name for residuals_white_dir. Use the Table_Save ECSV file from the optimized WLC S5 output dir 

In [ ]:
# manual_clip_lists = [[0, 26], [26, 27], [52, 53], [78, 79]]  # TEST - Standard
# manual_clip_lists = [[0, 7], [26, 27], [52, 53], [78, 79]]  # TEST - Visit36-39
# manual_clip_lists = [[0, 9], [26, 27], [52, 53], [78, 79]]  # TEST - Visit28-31

In [ ]:
## STEP 2 - Run Stages 3-6 using optimized values ##

print("Performing final run (Stage 3 - Stage 6).")

# ## Setup Meta ##
s3_meta_GA = readECF.MetaClass(ecf_path, s3_ecffile)
s4_meta_GA = readECF.MetaClass(ecf_path, s4_ecffile)
s5_meta_GA = readECF.MetaClass(ecf_path, s5_ecffile)
s6_meta_GA = readECF.MetaClass(ecf_path, s6_ecffile)

# # Load previously saved best S3-S5 ECF and EPF inputs
# with open(file_path, 'r') as f:
#     # json.load() loads the dictionary from a file
#     best = json.load(f)

# print(best) 


# # Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
# s3_meta_GA.xwindow = ev.xwindow
# s3_meta_GA.ywindow = ev.ywindow

# # Overwrite ECF values with extracted ev.xwindow and ev.ywindow values
s3_meta_GA.xwindow = [best['xwindow_LB'], best['xwindow_UB']]
# s3_meta_GA.ywindow = [best['ywindow_LB'], best['ywindow_UB']]
if "ywindow_LB" in best:
    s3_meta_GA.ywindow = [best['ywindow_LB'], best['ywindow_UB']]
else:
    best['ywindow_LB'] = ev.ywindow[0]
    best['ywindow_UB'] = ev.ywindow[1]
    s3_meta_GA.ywindow = [best['ywindow_LB'], best['ywindow_UB']]
    

# Use 27 spec channels (20 nm channel bw) for For HST WFC-3 G141 
s4_meta_GA.nspecchan = 27 

# Manual Clipping
s5_meta_GA.manual_clip = manual_clip_lists

# Fit Method
s5_meta_GA.fit_method = 'emcee'

# Divide out White Noise
s5_meta_GA.divide_white = True

# Hide Plots
s5_meta_GA.hide_plots = False

# Retrieve S6 ECF values from ExoMAST
s6_meta_GA.star_Rad = planet_data['Rs']
s6_meta_GA.planet_Teq = planet_data['Tp']
s6_meta_GA.planet_Mass = planet_data['Mp']

# s6_meta_GA.y_scalars = 100  # Show transit depth in percent rather than ppm

# Overwrite default meta inputs with optimized Stage 3 and Stage 4 inputs
# Stage 3
s3_meta_GA.diffthresh = best['diffthresh']
s3_meta_GA.bg_hw = best['bg_hw']
s3_meta_GA.spec_hw = best['spec_hw']
s3_meta_GA.p3thresh = best['p3thresh']
s3_meta_GA.median_thresh = best['median_thresh']
s3_meta_GA.window_len = best['window_len']
s3_meta_GA.p5thresh = best['p5thresh']
s3_meta_GA.p7thresh = best['p7thresh']
# Stage 4
s4_meta_GA.diffthresh = best['diffthresh']
s4_meta_GA.bg_hw = best['bg_hw']
s4_meta_GA.spec_hw = best['spec_hw']
s4_meta_GA.p3thresh = best['p3thresh']
s4_meta_GA.median_thresh = best['median_thresh']
s4_meta_GA.window_len = best['window_len']
s4_meta_GA.p5thresh = best['p5thresh']
s4_meta_GA.p7thresh = best['p7thresh']
s4_meta_GA.drift_range = best['drift_range']
s4_meta_GA.highpassWidth = best['highpassWidth']
s4_meta_GA.sigma = best['sigma']
s4_meta_GA.box_width = best['box_width']
# Stage 5
s5_meta_GA.diffthresh = best['diffthresh']
s5_meta_GA.bg_hw = best['bg_hw']
s5_meta_GA.spec_hw = best['spec_hw']
s5_meta_GA.p3thresh = best['p3thresh']
s5_meta_GA.median_thresh = best['median_thresh']
s5_meta_GA.window_len = best['window_len']
s5_meta_GA.p5thresh = best['p5thresh']
s5_meta_GA.p7thresh = best['p7thresh']
s5_meta_GA.drift_range = best['drift_range']
s5_meta_GA.highpassWidth = best['highpassWidth']
s5_meta_GA.sigma = best['sigma']
s5_meta_GA.box_width = best['box_width']

if enable_exotic_ld is True:

    # Retrieve Values for Exotic-ld
    s4_meta_GA.teff = planet_data['Teff']
    s4_meta_GA.logg = planet_data['stellar_gravity']
    s4_meta_GA.metallicity = planet_data['Fe/H']

    s5_meta_GA.teff = planet_data['Teff']
    s5_meta_GA.logg = planet_data['stellar_gravity']
    s5_meta_GA.metallicity = planet_data['Fe/H']

    # Turn on compute_ld
    s4_meta_GA.compute_ld = True

    # Turn on compute white
    s4_meta_GA.compute_white = True

    # Specify 1D or 3D Grid Model
    s4_meta_GA.exotic_ld_grid = '3D'

    # Path for exotic-ld ancillary files 
    s4_meta_GA.exotic_ld_direc = exotic_ld_direc

    # Path for exotic-ld throughput file 
    s4_meta_GA.exotic_ld_file = exotic_ld_file

    # Turn on use_generate_ld and enter paths for ld files
    s5_meta_GA.use_generate_ld = use_generate_ld

    # Path for ld file (white) 
    s5_meta_GA.ld_file = ld_file

    # Path for ld file (white) 
    s5_meta_GA.ld_file_white = ld_file_white


directory = s4_meta_GA.topdir
if not os.path.exists(directory):
    os.makedirs(directory)

s5_epffile = 's5_fit_par_' + eventlabel + '.epf'
    
# Call the Generate EPF function
# generate_epf.Stage_6(best, planet_data, file_path=os.path.join(directory, s5_epffile))


# Load params object          
params = readEPF.Parameters(s5_meta_GA.folder, s5_meta_GA.fit_par)

# Run Test
if __name__ == '__main__':
    s3_spec, s3_meta = s3.reduce(eventlabel, input_meta=s3_meta_GA)
    s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, input_meta=s4_meta_GA, s3_meta=s3_meta)
    s5_meta = s5.fitlc(eventlabel, params, input_meta=s5_meta_GA, s4_meta=s4_meta)
    s6_meta = s6.plot_spectra(eventlabel, input_meta=s6_meta_GA, s5_meta=s5_meta)

In [ ]:
# ## STEP 2 - Save Best S3-S5 ECF and EPF Values as .pkl ##

# # Use pickle.dump to save the dictionary
# with open("best_inputs.pkl", "wb") as f:
#     pickle.dump(best, f)

In [ ]:
# ## STEP 2 - Use pickle.load to load the "best" dictionary ##
# with open("best_inputs.pkl", "rb") as f:
#     best = pickle.load(f)

# print(best)  # prints: {'key1': 'value1', 'key2': 'value2'}

In [ ]:
## STEP 2 - Save results for optimized spectral data ##

outputdir_spec_S3 = s3_meta.outputdir
outputdir_spec_S4 = s4_meta.outputdir
outputdir_spec_S5 = s5_meta.outputdir
outputdir_spec_S6 = s6_meta.outputdir

# Assuming these are full path directories
directories = [
    (outputdir_spec_S3, "S3"),
    (outputdir_spec_S4, "S4"),
    (outputdir_spec_S5, "S5"),
    (outputdir_spec_S6, "S6")
]

dest_directory_spec = outputdir_optimization + "Spec"

# Copy files and subfolders
copy_files_and_subfolders_to_new_directory(directories, dest_directory_spec)

In [ ]:
## STEP 2 - Quality Assessment: Plot the chi2red per channel ##

# Initialize lists to store the channels and corresponding chi-squared values
channels = []
chi_squared_values = []

# Read the file
with open(s6_meta.outputdir + "S6_" + eventlabel + ".log", 'r') as f:
    log_content = f.read()
    
# Regular expression pattern to match the channel and chi-squared value
pattern = r'Starting Channel (\d+).*?Reduced Chi-squared: ([\d.]+)'

matches = re.findall(pattern, log_content, re.DOTALL)

# Append the results into the lists
for match in matches:
    channels.append(int(match[0]))
    chi_squared_values.append(float(match[1]))

# Create a pandas DataFrame
df = pd.DataFrame({
    'Channel': channels,
    'Chi_Squared': chi_squared_values
})

# df['Channel'] = df['Channel'] - 1

# Print the DataFrame
print(df)
chi2red_nspecchan = df

# Calculate median chi-squared
median_chi_squared = np.median(df['Chi_Squared'])

# Create a line plot
plt.figure(figsize=(10,6))
plt.plot(df['Channel'], df['Chi_Squared'], marker='o', label='Chi-squared value for channel')
plt.xlabel('Channel')
plt.ylabel(r'$\chi_{{red}}^2$')
plt.title('Reduced $\chi^2$ for each Channel')
plt.grid(True)

# Draw a horizontal line at the median value
plt.axhline(y=median_chi_squared, color='r', linestyle='--', label=f'Median $\chi_{{red}}^2$ = {median_chi_squared:.2f}')

# Add a legend
plt.legend(loc='best') # 'best' will choose the best location for the legend based on minimizing overlap with data

# Save the plot to a file
output_file_path = os.path.join(outputdir_optimization, "chi_squared_nspecchan.png")
plt.savefig(output_file_path)

# Show the plot on the screen
plt.show()

# After saving, it's good to close the plot to free up resources
plt.close()

In [ ]:
## STEP 2 - Create output file for optimization results ##

output_file = outputdir_optimization + "optimization_results.txt"

with open(output_file, 'w') as f:
    # # Write User Input Parameters
    # f.write("## INPUT PARAMETERS ##\n\n")

    # f.write("## Planet Name ##\n")
    # f.write(f"planet_name = '{planet_name}'\n")
    # f.write(f"eventlabel = '{eventlabel}'\n\n")

    # f.write("## Optimizer ##\n")
    # f.write(f"optimizer = '{optimizer}'\n\n")

    # f.write("## Genetic Algorithm ##\n")
    # f.write(f"target_fitness = {target_fitness}\n\n")

    # f.write("# GA - Step 1 Settings\n")
    # f.write(f"population_size_Step_1 = {population_size_Step_1}\n")
    # f.write(f"generations_Step_1 = {generations_Step_1}\n\n")

    # f.write("# GA - Step 2 Settings\n")
    # f.write(f"population_size_Step_2 = {population_size_Step_2}\n")
    # f.write(f"generations_Step_2 = {generations_Step_2}\n\n")

    # f.write("# GA - Step 3 Settings\n")
    # f.write(f"population_size_Step_3 = {population_size_Step_3}\n")
    # f.write(f"generations_Step_3 = {generations_Step_3}\n\n")

    # f.write("# Fitness Scaling\n")
    # f.write(f"scaling_chi2red = {scaling_chi2red}\n")
    # f.write(f"scaling_MAD_spec = {scaling_MAD_spec}\n")
    # f.write(f"scaling_MAD_white = {scaling_MAD_white}\n\n")

    # f.write("## Skip Steps ##\n")
    # f.write(f"skip_xwindow_crop = {skip_xwindow_crop}\n\n")

    # f.write("## Limb Darkening ##\n")
    # f.write(f"enable_exotic_ld = {enable_exotic_ld}\n\n\n")


    f.write("## OPTIMIZATION RESULTS ##\n\n")

    # Write Dictionary Contents
    f.write("## Optimized ECF & EPF Inputs ##\n")
    
    # Use best_white to ensure best params from an 
    # indiviual spec channel isnt used
    for key, value in best_white.items():
        f.write(f"{key}: {value}\n")

    f.write("\n")

    # Write Optimization Results
    f.write("## Optimization Metrics - General ##\n")
    f.write(f"MAD_WLC = {sum(s4_meta_best_WLC.mad_s4_binned) / len(s4_meta_best_WLC.mad_s4_binned)}\n")
    f.write(f"MAD_Spec = {s4_meta_best_WLC.mad_s4}\n")
    f.write(f"chi2red_WLC = {s5_meta_best_WLC.chi2red}\n\n")

    # Writing the chi2red DataFrame
    f.write("## Optimization Metrics - Channel Fits ##\n")
    chi2red_nspecchan.to_csv(f, sep='\t', index=False)
    f.write(f"Median chi2red = {median_chi_squared}\n")

print(f"Optimization results saved to {output_file}")